<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#必要な機能の呼び出し" data-toc-modified-id="必要な機能の呼び出し-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>必要な機能の呼び出し</a></span></li><li><span><a href="#初期設定" data-toc-modified-id="初期設定-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>初期設定</a></span></li><li><span><a href="#データ読み込み" data-toc-modified-id="データ読み込み-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>データ読み込み</a></span></li><li><span><a href="#クレンジング" data-toc-modified-id="クレンジング-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>クレンジング</a></span></li><li><span><a href="#モデル構築" data-toc-modified-id="モデル構築-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>モデル構築</a></span><ul class="toc-item"><li><span><a href="#最適化指標＝Accuracy" data-toc-modified-id="最適化指標＝Accuracy-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>最適化指標＝Accuracy</a></span></li><li><span><a href="#最適化指標＝AUC" data-toc-modified-id="最適化指標＝AUC-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>最適化指標＝AUC</a></span></li></ul></li><li><span><a href="#最終パラメータ決定--&gt;-モデル作成--&gt;-保存" data-toc-modified-id="最終パラメータ決定-->-モデル作成-->-保存-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>最終パラメータ決定 -&gt; モデル作成 -&gt; 保存</a></span></li></ul></div>

# 必要な機能の呼び出し

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression # ロジスティック回帰モデル


# 初期設定

In [13]:
root = os.path.abspath('../')

folders = ['data', 'models']
path = {}

for f_name in folders:
    path[f_name] = root.replace('\\', '/') + '/%s'%f_name

In [3]:
# 年齢クレンジング用の関数
def clns_age(age):
    age_cls = ''
    if age < 10:
        age_cls = 'lt10'
    elif age < 20:
        age_cls = 'lt20'
    elif age < 30:
        age_cls = 'lt30'
    elif age < 40:
        age_cls = 'lt40'
    elif age < 50:
        age_cls = 'lt50'
    elif age < 60:
        age_cls = 'lt60'
    elif age < 70:
        age_cls = 'lt70'
    else:
        age_cls = 'unclear'
    
    return age_cls
        

# データ読み込み

In [15]:
# 学習用データ
train = pd.read_csv(path['data'] + '/raw/train_enshu.tsv', sep='\t')

# テスト用データ
test = pd.read_csv(path['data'] + '/raw/test_enshu.tsv', sep='\t')


In [16]:
train.T

,0,1,2,3,4,5,6,7,8,9,...,335,336,337,338,339,340,341,342,343,344
id,57,571,737,721,551,291,739,143,292,348,...,581,371,296,324,751,429,117,604,442,617
survived,0,1,1,0,0,1,0,0,0,1,...,1,0,0,0,1,1,0,1,0,0
pclass,3,1,1,3,2,1,3,3,2,3,...,1,3,3,3,3,3,2,1,3,3
sex,male,female,male,male,male,female,male,male,male,male,...,female,male,male,male,male,male,male,male,male,female
age,28.5,53,35,17,27,19,NaN,19,36,3,...,39,18,23.5,NaN,6,32,29,35,25,26
sibsp,0,2,0,1,0,1,0,0,0,1,...,1,1,0,8,0,0,1,0,1,1
parch,0,0,0,0,0,0,0,0,0,1,...,1,0,0,2,1,0,0,0,0,0
fare,7.2292,51.4792,512.329,7.0542,26,91.0792,7.8958,6.75,12.875,15.9,...,110.883,6.4958,7.2292,69.55,12.475,8.05,21,26.55,7.775,16.1
embarked,C,S,C,S,S,C,S,Q,C,S,...,C,S,C,S,S,S,S,C,S,S


In [17]:
test.T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
id,849,67,355,280,781,189,88,552,134,149,...,774,393,757,430,797,427,504,582,591,148
survived,1,0,0,0,1,0,1,0,0,0,...,1,1,0,1,1,1,1,0,1,0
pclass,1,3,3,3,1,3,1,3,2,2,...,2,1,2,1,3,2,1,2,1,2
sex,female,male,male,male,female,male,female,male,male,male,...,female,female,male,male,female,female,female,male,female,male
age,NaN,19,28,65,17,36,23,NaN,25,42,...,54,23,18,28,31,19,16,54,52,36.5
sibsp,1,0,0,0,1,0,3,0,0,0,...,1,1,0,0,0,0,0,0,1,0
parch,0,0,0,0,0,0,2,0,0,0,...,3,0,0,0,0,0,0,0,0,2
fare,89.1042,8.1583,9.5,7.75,57,7.8958,263,7.8292,13,13,...,23,113.275,11.5,26.55,8.6833,26,86.5,26,78.2667,26
embarked,C,S,S,Q,S,S,S,Q,S,S,...,S,C,S,S,S,S,S,S,C,S


# クレンジング

In [18]:
# 学習データクレンジング -> 文字列のカラムを数値（フラグ化）
age_train = pd.get_dummies(train['age'].apply(clns_age))
embarked_train = pd.get_dummies(train.embarked)
gender_train = pd.get_dummies(train.sex)
train_tmp = train.drop(['id', 'sex', 'embarked', 'age'], axis=1)

# 学習データ整形
train_clns = pd.concat([train_tmp, embarked_train, gender_train, age_train], axis=1)

# 説明変数
train_X = train_clns.drop('survived', axis=1)

# 目的変数
train_y = train_clns['survived']


In [19]:
# テストデータクレンジング -> 文字列のカラムを数値（フラグ化）
age_test = pd.get_dummies(test['age'].apply(clns_age))
embarked_test = pd.get_dummies(test.embarked)
gender_test = pd.get_dummies(test.sex)
test_tmp = test.drop(['id', 'sex', 'embarked', 'age'], axis=1)

# テストデータ整形
test_clns = pd.concat([test_tmp, embarked_test, gender_test, age_test], axis=1)

# 説明変数
test_X = test_clns.drop('survived', axis=1)

# 目的変数
test_y = test_clns['survived']


# モデル構築

## 最適化指標＝Accuracy

In [20]:
# グリッドサーチ
# C = [1e-7,1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.,10.,100.,1000.,100000.]
penalty = ['l1', 'l2']
C = [i/10. for i in range(1,30)]

# パラメーターとスコア格納用リストを宣言
ps = []
cs = []
ss = []

for p in penalty:
    for c in C:
        
        print('==============================================================================================')
        print('+ Parameter')
        print('  - penalty = %s' % p)
        print('  - C = %s' % c)
        print()

        # 学習
        clf = LogisticRegression(penalty=p, C=c)
        clf.fit(X=train_X, y=train_y)

        # 予測
        pred = clf.predict(test_X)
        
        # 検証
        accuracy = clf.score(test_X, test_y)
        auc = roc_auc_score(test_y, clf.predict_proba(test_X)[: ,1])

        print('+ Score')
        print('  - Accuracy = %s' % accuracy)
        print('  - AUC      = %s' % auc)
        print()

        print('+ Report')
        print(classification_report(y_true=test_y, y_pred=pred))
        print()

        print('+ matrix')
        print(confusion_matrix(y_true=test_y, y_pred=pred))
        print()
        
        ps.append(p)
        cs.append(c)
        ss.append(accuracy)


+ Parameter
  - penalty = l1
  - C = 0.1

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9412121212121213

+ Report
             precision    recall  f1-score   support

          0       0.84      0.93      0.88        55
          1       0.90      0.78      0.83        45

avg / total       0.86      0.86      0.86       100


+ matrix
[[51  4]
 [10 35]]

+ Parameter
  - penalty = l1
  - C = 0.2

+ Score
  - Accuracy = 0.88
  - AUC      = 0.944040404040404

+ Report
             precision    recall  f1-score   support

          0       0.84      0.96      0.90        55
          1       0.95      0.78      0.85        45

avg / total       0.89      0.88      0.88       100


+ matrix
[[53  2]
 [10 35]]

+ Parameter
  - penalty = l1
  - C = 0.3

+ Score
  - Accuracy = 0.88
  - AUC      = 0.9509090909090909

+ Report
             precision    recall  f1-score   support

          0       0.84      0.96      0.90        55
          1       0.95      0.78      0.85        45

avg / to

+ Score
  - Accuracy = 0.86
  - AUC      = 0.956969696969697

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       100


+ matrix
[[53  2]
 [12 33]]

+ Parameter
  - penalty = l1
  - C = 2.6

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9565656565656565

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       100


+ matrix
[[53  2]
 [12 33]]

+ Parameter
  - penalty = l1
  - C = 2.7

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9565656565656565

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       1

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9571717171717172

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       100


+ matrix
[[53  2]
 [12 33]]

+ Parameter
  - penalty = l2
  - C = 2.7

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9571717171717172

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       100


+ matrix
[[53  2]
 [12 33]]

+ Parameter
  - penalty = l2
  - C = 2.8

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9571717171717172

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       

In [21]:
score = pd.DataFrame(
    {
        'penalty':ps,
        'C':cs,
        'accuracy':ss
    }
)

score[score.accuracy == score['accuracy'].max()]

,penalty,C,accuracy
1,l1,0.2,0.88
2,l1,0.3,0.88
3,l1,0.4,0.88


## 最適化指標＝AUC

In [22]:
# グリッドサーチ
# C = [1e-7,1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.,10.,100.,1000.,100000.]
penalty = ['l1', 'l2']
C = [i/10. for i in range(1,30)]

# パラメーターとスコア格納用リストを宣言
ps = []
cs = []
ss = []

for p in penalty:
    for c in C:
        
        print('==============================================================================================')
        print('+ Parameter')
        print('  - penalty = %s' % p)
        print('  - C = %s' % c)
        print()

        # 学習
        clf = LogisticRegression(penalty=p, C=c)
        clf.fit(X=train_X, y=train_y)

        # 予測
        pred = clf.predict(test_X)
        
        # 検証
        accuracy = clf.score(test_X, test_y)
        auc = roc_auc_score(test_y, clf.predict_proba(test_X)[: ,1])

        print('+ Score')
        print('  - Accuracy = %s' % accuracy)
        print('  - AUC      = %s' % auc)
        print()

        print('+ Report')
        print(classification_report(y_true=test_y, y_pred=pred))
        print()

        print('+ matrix')
        print(confusion_matrix(y_true=test_y, y_pred=pred))
        print()
        
        ps.append(p)
        cs.append(c)
        ss.append(auc)


+ Parameter
  - penalty = l1
  - C = 0.1

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9412121212121213

+ Report
             precision    recall  f1-score   support

          0       0.84      0.93      0.88        55
          1       0.90      0.78      0.83        45

avg / total       0.86      0.86      0.86       100


+ matrix
[[51  4]
 [10 35]]

+ Parameter
  - penalty = l1
  - C = 0.2

+ Score
  - Accuracy = 0.88
  - AUC      = 0.944040404040404

+ Report
             precision    recall  f1-score   support

          0       0.84      0.96      0.90        55
          1       0.95      0.78      0.85        45

avg / total       0.89      0.88      0.88       100


+ matrix
[[53  2]
 [10 35]]

+ Parameter
  - penalty = l1
  - C = 0.3

+ Score
  - Accuracy = 0.88
  - AUC      = 0.9509090909090909

+ Report
             precision    recall  f1-score   support

          0       0.84      0.96      0.90        55
          1       0.95      0.78      0.85        45

avg / to

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9575757575757575

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       100


+ matrix
[[53  2]
 [12 33]]

+ Parameter
  - penalty = l2
  - C = 0.3

+ Score
  - Accuracy = 0.85
  - AUC      = 0.9591919191919192

+ Report
             precision    recall  f1-score   support

          0       0.80      0.96      0.88        55
          1       0.94      0.71      0.81        45

avg / total       0.87      0.85      0.85       100


+ matrix
[[53  2]
 [13 32]]

+ Parameter
  - penalty = l2
  - C = 0.4

+ Score
  - Accuracy = 0.86
  - AUC      = 0.9583838383838383

+ Report
             precision    recall  f1-score   support

          0       0.82      0.96      0.88        55
          1       0.94      0.73      0.83        45

avg / total       0.87      0.86      0.86       

In [23]:
score = pd.DataFrame(
    {
        'penalty':ps,
        'C':cs,
        'auc':ss
    }
)

score[score.auc == score['auc'].max()]

,penalty,C,auc
16,l1,1.7,0.960606
17,l1,1.8,0.960606
18,l1,1.9,0.960606
19,l1,2.0,0.960606
20,l1,2.1,0.960606


# 最終パラメータ決定 -> モデル作成 -> 保存

In [24]:
clf = LogisticRegression(penalty='l1', C=1.9)
clf.fit(X=train_X, y=train_y)


LogisticRegression(C=1.9, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
joblib.dump(clf, path['models'] + '/titanic_suviver_clf_model_v02.pkl')


['C:/00_work/01_project/analysys_titanic/models/titanic_suviver_clf_model_v02.pkl']